<a href="https://colab.research.google.com/github/angiew-02/AHI-Capstone-Project/blob/main/AHI_Capstone_25'_Tableau_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing libraries
!pip install seaborn
!pip install pandas
!pip install numpy
!pip3 install pandas
!pip install matplotlib
!pip install xgboost
!pip install catboost
!pip install regex
!pip install scikit-learn
!pip install lightgbm
!pip install missingno
!pip install plotly.express

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
#import statsmodels.api as sm
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split  # For splitting the dataset into training and testing sets
from sklearn.linear_model import LinearRegression  # Linear regression model
from sklearn.metrics import mean_squared_error, r2_score  # Evaluation metrics
import matplotlib.pyplot as plt  # For data visualization

In [4]:
heartdisease = pd.read_csv('/content/Rates_and_Trends_in_Heart_Disease_and_Stroke_Mortality_Among_US_Adults__35___by_County__Age_Group__Race_Ethnicity__and_Sex___2000-2019 (1).csv')

/tmp/ipython-input-2485035719.py:1: DtypeWarning: Columns (0,12) have mixed types. Specify dtype option on import or set low_memory=False.
  heartdisease = pd.read_csv('/content/Rates_and_Trends_in_Heart_Disease_and_Stroke_Mortality_Among_US_Adults__35___by_County__Age_Group__Race_Ethnicity__and_Sex___2000-2019 (1).csv')


In [5]:
print(heartdisease.columns)

Index(['Year', 'LocationAbbr', 'LocationDesc', 'GeographicLevel', 'DataSource',
       'Class', 'Topic', 'Data_Value', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Confidence_limit_Low', 'Confidence_limit_High',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'LocationID'],
      dtype='object')


In [6]:
print(heartdisease.dtypes)

Year                           object
LocationAbbr                   object
LocationDesc                   object
GeographicLevel                object
DataSource                     object
Class                          object
Topic                          object
Data_Value                    float64
Data_Value_Unit                object
Data_Value_Type                object
Data_Value_Footnote_Symbol     object
Data_Value_Footnote            object
Confidence_limit_Low           object
Confidence_limit_High         float64
StratificationCategory1        object
Stratification1                object
StratificationCategory2        object
Stratification2                object
StratificationCategory3        object
Stratification3                object
LocationID                    float64
dtype: object


In [7]:
heartdisease.replace('?', np.nan, inplace=True)

In [8]:
heartdisease.drop_duplicates(inplace=True)

In [9]:
# Replace empty strings or strings with only whitespace with np.nan
heartdisease = heartdisease.replace(r'^\s*$', np.nan, regex=True)

In [10]:
columns_to_drop = [
     'LocationDesc', 'GeographicLevel', 'DataSource', 'Class','Data_Value_Unit', 'Data_Value_Type','Data_Value_Footnote','StratificationCategory1', 'StratificationCategory2','StratificationCategory3','LocationID','Data_Value_Footnote_Symbol'
]
# Drop only existing columns
existing_cols = [col for col in columns_to_drop if col in heartdisease.columns]
heartdisease_cleaned = heartdisease.drop(columns=existing_cols)
print(heartdisease_cleaned.head(10))

   Year LocationAbbr              Topic  Data_Value Confidence_limit_Low  \
0  1999           AL  All heart disease         NaN                  NaN   
1  2013           AL  All heart disease         NaN                  NaN   
2  2014           AL  All heart disease         NaN                  NaN   
3  2005           AL  All heart disease         NaN                  NaN   
4  2012           AL  All heart disease         NaN                  NaN   
5  2010           AL  All heart disease         NaN                  NaN   
6  2009           AL  All heart disease         NaN                  NaN   
7  2011           AL  All heart disease         NaN                  NaN   
8  2007           AL  All heart disease         NaN                  NaN   
9  2019           AL  All heart disease         NaN                  NaN   

   Confidence_limit_High   Stratification1                Stratification2  \
0                    NaN  Ages 35-64 years  American Indian/Alaska Native   
1        

In [11]:
heartdisease_cleaned.rename(columns={
    'Stratification1': 'Age',
    'Stratification2': 'Race',
    'Stratification3': 'Gender',
    'Topic':'Disease Type'
}, inplace=True)

In [12]:
# Data preview
heartdisease_cleaned.head(20)

,Year,LocationAbbr,Disease Type,Data_Value,Confidence_limit_Low,Confidence_limit_High,Age,Race,Gender
0,1999,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
1,2013,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
2,2014,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
3,2005,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
4,2012,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
5,2010,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
6,2009,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
7,2011,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
8,2007,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
9,2019,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall


In [13]:
print(heartdisease_cleaned.columns)

Index(['Year', 'LocationAbbr', 'Disease Type', 'Data_Value',
       'Confidence_limit_Low', 'Confidence_limit_High', 'Age', 'Race',
       'Gender'],
      dtype='object')


In [14]:
# Preview
heartdisease_cleaned.head(10)

,Year,LocationAbbr,Disease Type,Data_Value,Confidence_limit_Low,Confidence_limit_High,Age,Race,Gender
0,1999,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
1,2013,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
2,2014,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
3,2005,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
4,2012,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
5,2010,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
6,2009,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
7,2011,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
8,2007,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall
9,2019,AL,All heart disease,NaN,NaN,NaN,Ages 35-64 years,American Indian/Alaska Native,Overall


In [18]:
heartdisease_cleaned.to_csv('heartdisease_cleaned.csv', index=False)

In [19]:
from google.colab import files
files.download('heartdisease_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>